# Character-Level LSTM in PyTorch

In this notebook, I'll construct a character-level LSTM with PyTorch. The network will train character by character on some text, then generate new text character by character. As an example, I will train on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

First let's load in our required resources for data loading and model creation.

In [107]:
from typing import Tuple, Dict, Any, List
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F


In [104]:
# Define the CUDA device
Device: str = 'cuda' if torch.cuda.is_available() else 'cpu'

## Load in Data

Then, we'll load the Anna Karenina text file and convert it into integers for our network to use. 

### Tokenization

In the second cell, below, I'm creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
# open text file and read in data as `text`
text: str = ""
with open('data/anna.txt', 'r') as f:
    text = f.read()

In [5]:
# Sample from the text
text[:50]

'Chapter 1\n\n\nHappy families are all alike; every un'

Now we have the text, encode it as integers.

In [11]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers

# Create a tuple of unique characters in the text, including the punctuations
chars: Tuple[str, ...] = tuple(set(text))
# Creates and index - character mapping in both directions
int2char: Dict[int, str] = dict(enumerate(chars))
char2int: Dict[str, int] = {ch: ii for ii, ch in int2char.items()}
# Encode the characters of the text with their integer indices
encoded: np.ndarray = np.array([char2int[ch] for ch in text])

In [12]:
print(f'Encoded text array shape: {encoded.shape}')

Encoded text array shape: (1985223,)


Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [13]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see those same characters encoded as integers.

In [14]:
encoded[:100]

array([60, 47, 31, 72, 27, 13, 57, 82, 21, 43, 43, 43, 65, 31, 72, 72, 16,
       82, 73, 31, 75, 74, 66, 74, 13,  5, 82, 31, 57, 13, 82, 31, 66, 66,
       82, 31, 66, 74, 59, 13, 20, 82, 13,  4, 13, 57, 16, 82, 36, 53, 47,
       31, 72, 72, 16, 82, 73, 31, 75, 74, 66, 16, 82, 74,  5, 82, 36, 53,
       47, 31, 72, 72, 16, 82, 74, 53, 82, 74, 27,  5, 82, 34, 35, 53, 43,
       35, 31, 16, 41, 43, 43, 29,  4, 13, 57, 16, 27, 47, 74, 53])

## Pre-processing the data

As you can see in our char-RNN image above, our LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and *then* converted into a column vector where only it's corresponsing integer index will have the value of 1 and the rest of the vector will be filled with 0's. Since we're one-hot encoding the data, let's make a function to do that!


In [28]:
# Dummy code to understand a part of creating the one-hot encoding
temp = np.array([[1, 2, 3], [4, 5, 6]])
print(temp.shape)
np.multiply(*temp.shape)

(2, 3)


6

In [29]:
def one_hot_encode(arr: np.ndarray, n_labels: int) -> np.ndarray:
    """Encode the numeric text array with one-hot encoding

    :param arr: Numerically encoded array. The array can have more 
        than one domensions
    :type arr: np.ndarray
    :param n_labels: Number of characters
    :type n_labels: int
    """
    
    # Initialize the the one-hot encoded array with zeros. Here we are initializing a 
    # zeros array with rows equal to the number of characters in the passed in array 
    # and columns equal to the number of total characters. This would help in creating 
    # the one-hot encoded vector. In this case instead of column vectors each character 
    # would be represented by a one hot encoded row vector.
    one_hot: np.ndarray = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones. We select all the indices from axis 0 
    # which represents all characters from the passed in array and for selecting the 
    # the indices from axis 1, it uses the passed in array because numeric values in 
    # the passed in array actually indices from the char2int dictionary, that we have 
    # created above. And we set only those indices to 1.
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `n_seqs` (also refered to as "batch size" in other places). Each of those sequences will be `n_steps` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps i.e., the length of each sequence. Then, to get the total number of batches, $K$, we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ sequences. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **TODO:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [100]:
def get_batches(arr: np.ndarray, n_seqs: int, n_steps: int) -> Tuple[np.ndarray, np.ndarray]:
    '''Create a generator that returns batches of size
       (n_seqs, n_steps) from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    
    # Get the number of characters per batch
    batch_size: int = n_seqs * n_steps
    
    ## TODO: Get the number of batches we can make
    n_batches: int = len(arr) // batch_size
    
    ## TODO: Keep only enough characters to make full batches
    arr = arr[:(n_batches * batch_size)]
    
    ## TODO: Reshape into batch_size rows
    arr = np.reshape(arr, newshape=(n_seqs, -1))
    
    ## TODO: Make batches
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x: np.ndarray = arr[:, n:n+n_steps]

        # The targets, shifted by one
        y: np.ndarray = np.zeros_like(x)
        # This handling is in place because for the 
        # last batch we'd inevitably go to IndexError 
        # since we have not accounted for the extra column 
        # needed for the target. In that case we'd wrap 
        # i.e., use the first column from the actual 
        # array as the target.
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y
        yield x, y


In [101]:
batches = get_batches(arr=encoded, n_seqs=10, n_steps=50)
count = 0
for x, y in batches:
    assert(x.shape == (10, 50))
    assert(y.shape == (10, 50))
    count += 1
print(f'Total number of batches: {count}')

Total number of batches: 7940


### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 10 and 50 sequence steps.

In [102]:
batches = get_batches(arr=encoded, n_seqs=10, n_steps=50)
x, y = next(batches)

In [103]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[60 47 31 72 27 13 57 82 21 43]
 [82 31 75 82 53 34 27 82 68 34]
 [ 4 74 53 41 43 43 61 56 13  5]
 [53 82 38 36 57 74 53 68 82 47]
 [82 74 27 82 74  5 30 82  5 74]
 [82 23 27 82 35 31  5 43 34 53]
 [47 13 53 82  3 34 75 13 82 73]
 [20 82 33 36 27 82 53 34 35 82]
 [27 82 74  5 53 64 27 41 82 14]
 [82  5 31 74 38 82 27 34 82 47]]

y
 [[47 31 72 27 13 57 82 21 43 43]
 [31 75 82 53 34 27 82 68 34 74]
 [74 53 41 43 43 61 56 13  5 30]
 [82 38 36 57 74 53 68 82 47 74]
 [74 27 82 74  5 30 82  5 74 57]
 [23 27 82 35 31  5 43 34 53 66]
 [13 53 82  3 34 75 13 82 73 34]
 [82 33 36 27 82 53 34 35 82  5]
 [82 74  5 53 64 27 41 82 14 47]
 [ 5 31 74 38 82 27 34 82 47 13]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

---
## Defining the network with PyTorch

Below is where you'll define the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `dropout_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic LSTM cell as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial cell state of all zeros. This is done like so

```python
self.init_weights()
```

In [122]:
class CharRNN(nn.Module):

    FORWARD_RETURN_TYPE = Tuple[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]
    PREDICT_RETURN_TYPE = Tuple[str, Tuple[torch.Tensor, torch.Tensor]]

    lstm: nn.LSTM
    dropout: nn.Dropout
    fc: nn.Linear
    drop_prob: float
    n_in: int
    n_out: int
    n_layers: int
    n_hidden: int
    lr: float
    chars: Tuple[str, ...]
    int2char: Dict[int, str]
    char2int: Dict[str, int]
    
    def __init__(self, tokens: Tuple[str, ...], n_steps: int=100, 
                n_hidden: int=256, n_layers: int=2, 
                drop_prob: float=0.5, lr: float=0.001) -> None:
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # Creating character dictionaries. Here we are doing pretty 
        # much the same thing, which we did before.
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        self.n_in, self.n_out = len(self.chars), len(self.chars)
        
        ## TODO: define the LSTM, self.lstm
        self.lstm = nn.LSTM(input_size=self.n_in, hidden_size=self.n_hidden, 
                            num_layers=self.n_layers, dropout=drop_prob, batch_first=True) 
        
        ## TODO: define a dropout layer, self.dropout
        self.dropout = nn.Dropout(p=self.drop_prob)
        
        ## TODO: define the final, fully-connected output layer, self.fc
        self.fc = nn.Linear(in_features=self.n_hidden, out_features=self.n_out)
        
        # initialize the weights
        self.init_weights()
      
    
    def forward(self, x: torch.Tensor, 
                hc: Tuple[torch.Tensor, torch.Tensor]) -> FORWARD_RETURN_TYPE:
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hc`. '''
        
        ## TODO: Get x, and the new hidden state (h, c) from the lstm
        x, (h, c) = self.lstm(x, hc)
        
        ## TODO: pass x through a droupout layer
        x = self.dropout(x)
        
        # Stack up LSTM outputs using view
        x = x.reshape(x.size()[0] * x.size()[1], self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        x = self.fc(x)
        
        # return x and the hidden state (h, c)
        return x, (h, c)
    
    
    def predict(self, char: Tuple[str, ...], h: Tuple[torch.Tensor, torch.Tensor]=None, top_k: int=None) -> PREDICT_RETURN_TYPE:
        ''' Given a character, predict the next character.
        
            Returns the predicted character and the hidden state.
        '''
        # Put the model to cuda
        self.to(device=Device)
        
        if h is None:
            h = self.init_hidden(n_seqs=1)
        
        x: np.ndarray = np.array([[self.char2int[char]]])
        x = one_hot_encode(arr=x, n_labels=len(self.chars))
        inputs: torch.Tensor = torch.from_numpy(x)
        h = tuple([each.data for each in h])
        out, h = self.forward(x=inputs.to(device=Device), hc=h)

        p: torch.Tensor = F.softmax(out, dim=1).data
        
        top_ch: np.ndarray = np.array([])
        if top_k is None:
            top_ch = np.arange(len(self.chars))
        else:
            p, top_ch = p.cpu().topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p: np.ndarray = p.numpy().squeeze()
        # top_ch is an array if indices and p is the array 
        # of corresponding probabilities. In the next line we 
        # choose a single index randomly and probability 
        # distribution for doing the same is computed by 
        # p/p.sum().
        char: int = np.random.choice(top_ch, p=p/p.sum())
        return self.int2char[char], h
    
    def init_weights(self) -> None:
        ''' Initialize weights for fully connected layer '''
        initrange = 0.1
        
        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0).to(device=Device)
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1).to(device=Device)
        
    def init_hidden(self, n_seqs: int) -> Tuple[torch.Tensor, torch.Tensor]:
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (weight.new(self.n_layers, n_seqs, self.n_hidden).zero_().to(device=Device),
                weight.new(self.n_layers, n_seqs, self.n_hidden).zero_().to(device=Device))
        

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

In [127]:
def train(net: CharRNN, data: np.ndarray, epochs: int = 10, n_seqs: int = 10, n_steps: int = 50, lr: float = 0.001, clip: int = 5, val_frac: float = 0.1, print_every: int = 10) -> None:
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        n_seqs: Number of mini-sequences per mini-batch, aka batch size
        n_steps: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    '''
    
    net.train()
    opt: torch.optim.Adam = torch.optim.Adam(net.parameters(), lr=lr)
    criterion: nn.CrossEntropyLoss = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx: int = int(len(data)*(1-val_frac))
    train_data, val_data = data[:val_idx], data[val_idx:]
    
    counter: int = 0
    n_chars: int = len(net.chars)

    for e in range(epochs):
        h: Tuple[torch.Tensor, torch.Tensor] = net.init_hidden(n_seqs=n_seqs) # Initialize the hidden state of the LSTM layers
        for x, y in get_batches(train_data, n_seqs, n_steps):
            
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            inputs, targets = inputs.to(device=Device), targets.type(torch.LongTensor).to(device=Device)

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net.forward(x=inputs, hc=h)
            loss: torch.Tensor = criterion(output, targets.view(n_seqs*n_steps))

            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            opt.step()
            
            if counter % print_every == 0:
                
                # Get validation loss
                val_h: Tuple[torch.Tensor, torch.Tensor] = net.init_hidden(n_seqs=n_seqs)
                val_losses: List[float] = []
                for x, y in get_batches(val_data, n_seqs, n_steps):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(arr=x, n_labels=n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    inputs, targets = inputs.to(device=Device), targets.type(torch.LongTensor).to(device=Device)

                    output, val_h = net.forward(x=inputs, hc=val_h)
                    val_loss: torch.Tensor = criterion(output, targets.view(n_seqs*n_steps))
                
                    val_losses.append(val_loss.cpu().item())
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Time to train

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes (number of sequences and number of steps), and start the training. With the train function, we can set the number of epochs, the learning rate, and other parameters. Also, we can run the training on a GPU by setting `cuda=True`.

In [124]:
if 'net' in locals():
    del net

In [125]:
# define and print the net
net = CharRNN(tokens=chars, n_hidden=512, n_layers=2).to(device=Device)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [128]:
n_seqs: int = 128  # Batch size
n_steps: int = 100 # Length of the sequence / time steps

# Open the training log file.
log_file = 'training_log.txt'
f = open(log_file, 'w')

# TRAIN
train(net, encoded, epochs=10, n_seqs=n_seqs, n_steps=n_steps, lr=0.001, print_every=10)

# Close the training log file.
f.close()


Epoch: 1/10... Step: 10... Loss: 3.2853... Val Loss: 3.2915
Epoch: 1/10... Step: 20... Loss: 3.2039... Val Loss: 3.1880
Epoch: 1/10... Step: 30... Loss: 3.0765... Val Loss: 3.0628
Epoch: 1/10... Step: 40... Loss: 2.8815... Val Loss: 2.8954
Epoch: 1/10... Step: 50... Loss: 2.6955... Val Loss: 2.7251
Epoch: 1/10... Step: 60... Loss: 2.6383... Val Loss: 2.6194
Epoch: 1/10... Step: 70... Loss: 2.5403... Val Loss: 2.5509
Epoch: 1/10... Step: 80... Loss: 2.4601... Val Loss: 2.5032
Epoch: 1/10... Step: 90... Loss: 2.4436... Val Loss: 2.4619
Epoch: 1/10... Step: 100... Loss: 2.4436... Val Loss: 2.4295
Epoch: 1/10... Step: 110... Loss: 2.3259... Val Loss: 2.3949
Epoch: 1/10... Step: 120... Loss: 2.3184... Val Loss: 2.3661
Epoch: 1/10... Step: 130... Loss: 2.3084... Val Loss: 2.3386
Epoch: 1/10... Step: 140... Loss: 2.2679... Val Loss: 2.3136
Epoch: 1/10... Step: 150... Loss: 2.2421... Val Loss: 2.2933
Epoch: 1/10... Step: 160... Loss: 2.1974... Val Loss: 2.2655
Epoch: 1/10... Step: 170... Loss:

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `n_seqs` - Number of sequences running through the network in one pass.
* `n_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [129]:
# change the name, for saving multiple files
model_name = 'rnn_1_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

## Sampling

Now that the model is trained, we'll want to sample from it. To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### Top K sampling

Our predictions come from a categorcial probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text.

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [133]:
def sample(net: CharRNN, size: int, prime: str = 'The', top_k: int = None) -> str:
    """Sample random text using the network"""
    net.to(device=Device)

    net.eval() # We don't want to compute the gradients
    
    # First off, run through the prime characters
    chars: List[str] = [ch for ch in prime]
    h: Tuple[torch.Tensor, torch.Tensor] = net.init_hidden(1)
    for ch in prime:
        char, h = net.predict(char=ch, h=h, top_k=top_k)
    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = net.predict(char=chars[-1], h=h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [135]:
print(sample(net=net, size=2000, prime='Anna', top_k=5))

Anna, went out of the room
when she said to him, and stopped, and set hope of so mill. The
charm of the same times taking them to his
forming, saw the people seamed," said Varenka.

"Well, I don't know him."

"Yes, but I see that I have thought then a long time, and was asking to the court of this same things at tone. All the minute the strange whitered
him, he waited the best more than and went on into her find heart in he would say all the moment they was simply, and he commedied
into a society. His face was the forest of the cloud with words the stair, and had to talk to him and had so mare for her family,
who, had told her over
his brother--he strickened and standing at the political significance was
the balletions, her hand out.

"Yes, if I've careful in the corn in the silence, when he did not know what was necessary for her, but
such attoness.

"Oh, if it was not in the state.

"Yes, I should do anything, and we shall see them at in anything that has not to be said to Anna's fac

## Loading a checkpoint

In [136]:
# Here we have loaded in a model that trained over 1 epoch `rnn_1_epoch.net`
with open('rnn_1_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [137]:
# Change cuda to True if you are using GPU!
print(sample(net=loaded, size=2000, top_k=5, prime="And Levin said"))

And Levin said to her, taking his earor. He saw he went about and a contention.

"Yes, I am all the more and the life of this?
It appears, and there was a money
and so seeing all that it had to
supper the sake, and the stares already forgetten his horse and a senden of the meeting, he tried to say, when she was a stirnd out
of her husband, and her company was not treed to the carry with a most
difficult to despair and at the man, they were
several talked about the mother, to see him, and so that he did not see him. She did not see her talk and three him the following his.

"I don't know it."

All the sick man who had so liked that it seemed to her that it seemed
a man of the legs. He saw that he was struggled in the cloths, and had been dinned at their subjects, and say, who
was surprised at the tenter of a love of hope and sat down to his finating, and at
that most divarted him to the mendal."

"Why soon," said Anna interested in the party to
the sack. And she had strong still sounded